### MDP
![alt bandit](../images/mdp.png)

<div dir="rtl">
    محیط هایی مثل bandit برای تعریف خیلی از مسایل کافی نیستن و زمانیکه action بر محیط تاثیر داشته باشه  یا state transition  صورت بگیره احتیاج داریم در تصمیم گیری های agent احتمالات بیشتری رو در نظر بگیریم چرن انتخاب یک action همیشه optimal نیست
برای حل این نوع از مسایل MDP تعریف میکنیم که اصطلاحا formalizing problem as MDP گفته میشه و شامل 

- rewards
- actions
- states
</div>
    
<div dir="rtl">  
agent در زمان t با انتخاب action_t در state_t یک reward_t+1 دریافت میکنه و به state_t+1 وارد میشه و محیط وارد مرحله متفاوتی میشه که انتخاب action قبل ممکنه بهترین انتخاب نباشه و احتیاج به محاسبات جدیدی داره تا با کمک سیگنال reward تصمیمات خودش رو update کنه.
    برای هر state میتونیم با انتخاب action های مختلف به state های مختلفی وارد بشیم و از اونجا که محیط تصادفی non-stationary هست باید احتمالات رو هم در نظر بگیریم، تعریف MDP در واقع همون state machine هست که برای رفتن به state ها احتمال های مختلفی داریم state-transition probability
</div>
    
<div dir="ltr">  
p(s_t+1, r_t+1 | s_t, a_t)
</div>
    
![alt bandit](../images/mdp_example.png)
    


<div dir="rtl"> در این محیط های sequential و dynamic که با MDP تعریف میشن  یک خصوصیت مشترک داریم به نام Markov property، به این معنی که agent احتیاج به دونستن همه اتفاقات گذشته و تصمیماتی که از آغاز تا الآن داشته نداره و تنها دونستن ۱ time-step قبل کافیه
</div>
    
### Goal and Reward

<div dir="rtl">
هدف اصلی همچنان جستجو برای دریافت بالاترین expected return در محیط هست، و تنها توجه کردن به immediate reward کافی نیست و باید عواقب هر action رو در طولانی مدت future reward در نظر بگیریم و با کمک gradient update به سمتی حرکت کنیم که مقدار loss کمتر و در نتیجه به optimal policy نزدیک بشیم
</div>
<div dir="rtl">
محیط ها رو میتونیم به ۲ دسته کلی تقسیم کنیم episodic و continuous که تفاوت عمده اونها در داشتن terminal state هست. محیط episodic دارای یک سری independent episode هست که بعد از اتمام هرepisode محیط و agent reset میشن به حالت اولیه. محیط continuous ولی قابلیت شکسته شدن به epiode هارو نداره.
ولی برای محیطی که هیچوقت terminate نمیشه چجوری expected future return تعریف میشه؟ این مشکل رو با یک تعریف جدید دیگه به نام discount factor (gamma) حل میکنیم. ایده اصلی اینه که ما با یه ضریبی تاثیر time-step های آینده رو کنترل میکنیم به این صورت که reward های زمان های نزدیکتر مهمتر باشن تا زمان های خیلی دور در آینده به این expected discounted return . میشه اثبات کرد که برای gamma کمتر از ۱ این return متنهایی finite  هست
</div>

<div dir="ltr">
G_t = 1/(1-gamma)
</div>

<div dir="rtl">
 در خیلی از محیط های شبیه سازی شده max-step تعریف میشه که با کمک اون میشه agent رو وادار کرد تصمیمات بهتری در زمان محدود بگیره
</div>




### GridWorld Random Policy 

In [2]:
import sys
import numpy as np
sys.path.insert(1, '../../reinforcement_learning/code')
from env.gridworld_env import GridWorldEnv
from matplotlib import pyplot as plt
grid_width = 5
grid_height = 5
env = GridWorldEnv(grid_width, grid_height)
action_probability = 0.25
discount_factor = 0.9

value = np.zeros((grid_width, grid_height))
while True:
    new_value = np.zeros_like(value)
    for i in range(grid_height):
        for j in range(grid_width):
            for action in env.action_space:
                (ni, nj), reward = env.step(state=[i, j], action=action)
                new_value[i, j] += action_probability * (reward + discount_factor * value[ni, nj])
    
    # continue until it reaches tability
    diff = np.sum(np.abs(value - new_value))
    if diff < 1e-4:
        for r in value:
            for val in r:
                print("%.1f \t" % round(val, 1), end=" ")
            print('\n')
        break
    value = new_value

3.3 	 8.8 	 4.4 	 5.3 	 1.5 	 

1.5 	 3.0 	 2.3 	 1.9 	 0.5 	 

0.1 	 0.7 	 0.7 	 0.4 	 -0.4 	 

-1.0 	 -0.4 	 -0.4 	 -0.6 	 -1.2 	 

-1.9 	 -1.3 	 -1.2 	 -1.4 	 -2.0 	 



### GridWorld Optimal Policy

In [3]:
grid_width = 5
grid_height = 5
env = GridWorldEnv(grid_width, grid_height)
action_probability = 0.25
discount_factor = 0.9

value = np.zeros((grid_width, grid_height))
while True:
    new_value = np.zeros_like(value)
    for i in range(grid_height):
        for j in range(grid_width):
            action_values = []
            for action in env.action_space:
                (ni, nj), reward = env.step(state=[i, j], action=action)
                action_values.append(reward + discount_factor * value[ni, nj])
            new_value[i][j] = np.max(action_values)
    # continue until it reaches tability
    diff = np.sum(np.abs(value - new_value))
    if diff < 1e-4:
        for r in value:
            for val in r:
                print("%.1f \t" % round(val, 1), end=" ")
            print('\n')
        break
    value = new_value

22.0 	 24.4 	 22.0 	 19.4 	 17.5 	 

19.8 	 22.0 	 19.8 	 17.8 	 16.0 	 

17.8 	 19.8 	 17.8 	 16.0 	 14.4 	 

16.0 	 17.8 	 16.0 	 14.4 	 13.0 	 

14.4 	 16.0 	 14.4 	 13.0 	 11.7 	 

